# Master Thesis Code by Anne Langerak (Erasmus University Rotterdam)

## JAL: an algebra for JSON Query Optimization 

-----------------------------------------------------------------------------------------------------------------------------------------

## JSON Data Model 

In [1]:
# Packages and Modules
import json
import random
import re
from operator import itemgetter, attrgetter

In [2]:
# Class Node, required for the node creation in a JSON tree
class Node:
    
    def __init__(self, v, t, i): 
        """ This function creates a node object"""
        if t == "atomic":
            self.value = v
        else:
            # if the node is of complex type the value is simply equal to the identifier
            self.value = i
        self.outgoingEdge = []
        self.type = t
        self.identifier = i
    
    
    def set_outgoingEdge(self, edge):
        """ This function updates the outgoingEdge property given some edge"""
        self.outgoingEdge.append(edge)
    

In [3]:
# Class Edge, required for the edge creation in a JSON tree
class Edge:
    
    def __init__(self, l, p, c, t, i, path):
        """ This function creates an edge object"""
        self.label = l
        self.parent = p
        self.child = c
        self.type = t
        self.identifier = i
        self.path = path 
    

In [4]:
# Class Tree, required for the creation of a JSON tree
class Tree:
    
    def __init__(self, top): 
        """ This function initializes a JSON tree by means of a root node"""
        self.root = top
        self.nodes = [top]
        self.edges = []
        self.node_count = 1
        self.edge_count = 0
    
    
    def get_nodes(self): 
        """ This function returns all the nodes currently present in the tree"""
        node_values = []
        for node in self.nodes:
            node_values.append(node.value)
        return node_values
    
    
    def get_edges(self):
        """ This function returns all the edges currently present in the tree"""
        edge_labels = []
        for edge in self.edges:
            edge_labels.append(edge.label)
        return edge_labels
    
    def tree_in_document(self):
        """ This function converst a tree into an object type representation"""
        nodes = {}
        edges = {}
        labels = {}
        
        for node in self.nodes:
            temp_outgoing_list = []
            for e in node.outgoingEdge:
                temp_outgoing_list.append(e.identifier)
            nodes[node.identifier] = {
                "value" : node.value,
                "outgoingEdge" : temp_outgoing_list,
                "type" : node.type,
            }
        
        for edge in self.edges:
            edges[edge.identifier] = {
                "label" : edge.label,
                "parent" : edge.parent.identifier,
                "child" : edge.child.identifier,
                "type" : edge.type, 
                "path": edge.path
            }
            
            if edge.path in labels.keys(): 
                labels[edge.path] = labels.get(edge.path) + [edge.child.identifier]
            else:
                labels[edge.path] = [edge.child.identifier] 
        
        json_doc = {
                    "nodes" : nodes,
                    "edges" : edges,
                    "labels" : labels
                   }
        return json_doc
    
        

In [5]:
def addition(tree, parent, child, key):
    """ This function creates nodes and/ or edges based on the input in the given tree"""
    type_child = type(child)

    if type_child == list or type_child == 'list' or isinstance(type_child, list): # node is of type list (complex)
        child_node = Node(child, "complex", tree.node_count + 1) # create node object
        edge = Edge(key, parent, child_node, "str", tree.edge_count + 1, key) # create edge object
        Node.set_outgoingEdge(parent, edge) # set outgoing edge for the node
        # update tree's counters
        tree.nodes.append(child_node)
        tree.node_count += 1
        tree.edges.append(edge)
        tree.edge_count += 1
            
        for i in range(0, len(child)): # add all child nodes of the node to the tree
            sub_child = child[i]
            type_sub_child = type(sub_child)
            
            if  type_sub_child in [bool, str, int, float]: # the subchild is of atomic type
                sub_child_node = Node(sub_child, "atomic", tree.node_count + 1)
                edge_path = key 
                sub_edge = Edge(i+1, child_node, sub_child_node, "int", tree.edge_count + 1, edge_path)
                Node.set_outgoingEdge(child_node, sub_edge)
                tree.nodes.append(sub_child_node)
                tree.node_count += 1
                tree.edges.append(sub_edge)
                tree.edge_count += 1
        
            else: # it is of type list or dictionary (complex)
                sub_child_node = Node(sub_child, "complex", tree.node_count + 1)
                edge_path = key 
                sub_edge = Edge(i+1, child_node, sub_child_node, "int", tree.edge_count + 1, edge_path)
                Node.set_outgoingEdge(child_node, sub_edge)
                tree.nodes.append(sub_child_node)
                tree.node_count += 1
                tree.edges.append(sub_edge)
                tree.edge_count += 1
                
                if type_sub_child == dict: # subchild is an object
                    # key traversal
                    for subkey in sub_child.keys():
                        addition(tree, sub_child_node, sub_child.get(subkey), (edge_path + "." + subkey))
                else: # subchild is of type list
                    # index traversal
                    for j in range(0, len(sub_child)):
                        addition(tree, sub_child_node, sub_child[j], (edge_path))
                        
    if type_child == dict: # node is of type dictionary (complex)
        child_node = Node(child, "complex", tree.node_count + 1)

        edge = Edge(key, parent, child_node, "str", tree.edge_count + 1, key)
        Node.set_outgoingEdge(parent, edge)
        tree.nodes.append(child_node)
        tree.node_count += 1
        tree.edges.append(edge)
        tree.edge_count += 1
        parent_edge_label = key
        
        for key in child.keys(): # key traversal over the node
            sub_child = child.get(key)
            type_sub_child = type(sub_child)
            
            if  type_sub_child in [bool, str, int, float] or sub_child is None:
                sub_child_node = Node(sub_child, "atomic", tree.node_count + 1)
                sub_edge = Edge(key, child_node, sub_child_node, "str", tree.edge_count + 1, parent_edge_label + "." + str(key))
                Node.set_outgoingEdge(child_node, sub_edge)
                tree.nodes.append(sub_child_node)
                tree.node_count += 1
                tree.edges.append(sub_edge)
                tree.edge_count += 1
                
            else: # it is of type list or dictionary
                sub_child_node = Node(sub_child, "complex", tree.node_count + 1)
                sub_edge = Edge(key, child_node, sub_child_node, "str", tree.edge_count + 1, parent_edge_label + "." + str(key))
                Node.set_outgoingEdge(child_node, sub_edge)
                tree.nodes.append(sub_child_node)
                tree.node_count += 1
                tree.edges.append(sub_edge)
                tree.edge_count += 1
                
                if type_sub_child == dict: 
                    # key traversal
                    for subkey in sub_child.keys():
                        addition(tree, sub_child_node, sub_child.get(subkey), (parent_edge_label + "." + str(key) + "." + str(subkey)))
                else: # it is of type list
                    # index traversal
                    for j in range(0, len(sub_child)):
                        addition(tree, sub_child_node, sub_child[j], parent_edge_label + "." + str(key))# + "." + str(j+1))
    
    elif type_child in [str, bool, int, float]: # node is of atomic type
        child_node = Node(child, "atomic", (tree.node_count + 1))
        if type(key) == str:
            edge = Edge(key, parent, child_node, "str", tree.edge_count + 1, key)
        else:
            edge = Edge(key, parent, child_node, "int", tree.edge_count + 1, key)
        Node.set_outgoingEdge(parent, edge)
        tree.nodes.append(child_node)
        tree.node_count += 1
        tree.edges.append(edge)
        tree.edge_count += 1
                    

def json_datamodel(C, c_name): 
    """ This functions converts a collection of JSON documents into a collection of JSON  trees"""
    collection_of_trees = []
    
    for i in range(0, len(C)): # iterate over JSON documents
        root_node = Node("root", "complex", 1) # create root node
        tree = Tree(root_node) # initialize tree object
    
        document = C[i]

        for key in document.keys(): # iteratore over the (first-level) keys in JSON document
            addition(tree, root_node, document.get(key), c_name +"."+ str(key))
        
        collection_of_trees.append(tree.tree_in_document())
    
    return collection_of_trees
        

# Algebraic Operators

### Construction operators 

**Node**

In [6]:
def add_node(tree, node_type, node_value=None):  
    """ This function adds a new node to an existing tree (Note that only node of atomic values are possible in this current implementation, can easily be extended)"""
    num_nodes = len(tree.get('nodes'))
    if node_value == None:
        node_value = num_nodes + 1
    
    existing_nodes = tree.get('nodes')
    existing_nodes[num_nodes + 1] = {'value': node_value, 'outgoingEdge': [], 'type': node_type}
    
    tree['nodes'] = existing_nodes
    
    return tree

**Edge**

In [7]:
def add_edge(tree, parent, child, edge_type, label=None,):    
    """ This function adds a new edge to an existing tree"""
    edge_identifier = len(tree.get('edges')) + 1
    if label == None: # label is not specified, hence it should be of type int
        present_label = edge_identifier
    else: 
        present_label = label
        
    existing_edges = tree.get('edges')
    existing_edges[str(edge_identifier+1)] = {'label': present_label, 'parent': parent, 'child': child, 'type': edge_type}
    tree["edges"] = existing_edges
    
    # update parents outgoingedge identifiers
    existing_nodes = tree.get('nodes')
    node_info = existing_nodes.get(parent)
    existing_nodes[str(parent)] = {'value': node_info.get('value'), 'outgoingEdge': node_info.get('outgoingEdge').append(edge_identifier), 'type': node_info.get('type')}
    tree["nodes"] = existing_nodes
    
    # update labels with childnode identifier
    existing_labels = tree.get('labels')
    if present_label in existing_labels.keys():
        existing_labels[present_label] = existing_labels.get(present_label).append(child)
    else:
        existing_labels[present_label] = [child]
    tree["labels"] = existing_labels    
    
    return tree


### Extraction Operators 

**Projection operators**: 

In [8]:
def projection(C, labels):
    """ This function only projects the specified labels for each JSON document from the collection"""
    output = []
    
    for model in C:
        result_nodes = {}
        result_edges = {}
        result_labels = {}
        
        relevant_nodes = []
        labels_requested_not_present = []
        
        existing_edges = model.get('edges')

        # finding relevant nodes
        for edge_ident in existing_edges.keys():
            edge_info = existing_edges.get(edge_ident)
            edge_label = edge_info.get('path')

            if edge_label in labels:
                result_edges[edge_ident] = edge_info
                relevant_nodes = relevant_nodes + [edge_info.get('child')]
        

        existing_nodes = model.get('nodes')
        # for each relevant add its descendants (edges as well as nodes to the result)
        while relevant_nodes != []:
            for node in relevant_nodes:
                node_information = existing_nodes.get(node)
                result_nodes[node] = node_information

                for out_edge in node_information.get('outgoingEdge'):
                    out_edge_info = existing_edges.get(out_edge)
                    result_edges[out_edge] = out_edge_info 
                    child = out_edge_info.get('child')
                    if child is not None:
                        relevant_nodes = relevant_nodes + [child]
                relevant_nodes = relevant_nodes[1:]
        
        if len(result_nodes) == 0: # no relevant nodes
            continue
        
        existing_labels = model.get("labels")
        for label in labels:
            result_labels[label] = existing_labels.get(label)
        
        output.append({
                        "nodes" : result_nodes,
                        "edges" : result_edges,
                        "labels": result_labels
                        }) 
    return output




**Selection operator**

In [9]:
def selection(C, conditions):  
    """ This function performs one or more selections on each document from the collection"""
    
    def regex_condition(cd):
        """ This function reads the condition(s) and transforms it to machine readable instruction"""
        desired_operator = re.findall(r'>=|<=|!=|=|<|>', cd)[0]
        variable_names = re.split(r'>=|<=|!=|=|<|>', cd)
        
        if desired_operator == "=": # python syntax
            desired_operator = "=="
            
        condition_part1 = re.split(r'\$', variable_names[0])[-1].strip("'").strip('"').rstrip().lstrip().strip("`").strip("'").replace("'", "")
        temp_varia_1 = variable_names[1].lstrip().rstrip()
        temp_co_2 = re.split(r'\$', temp_varia_1)
        condition_part2 = temp_co_2[-1].strip("'").strip('"').rstrip().lstrip().strip("`").strip("'").strip("`").strip("'").replace("'", "")

        if "$" not in temp_varia_1: # comparison with a value 
            if condition_part2.isnumeric():
                if "." not in condition_part2 and not "," in condition_part2:
                    rewritten_condition = "vertex.get('nodes').get(vertex.get('labels').get('{0}')[0]).get('value') ".format(condition_part1) + " " + str(desired_operator) + " int({0}) ".format(condition_part2)
                else:
                    rewritten_condition = "vertex.get('nodes').get(vertex.get('labels').get('{0}')[0]).get('value') ".format(condition_part1) + " " + str(desired_operator) + " float({0}) ".format(condition_part2)
            else:
                if desired_operator == "!=": # switch condition parts
                    search_value = " '{0}' ".format(condition_part2)
                    rewritten_condition = "vertex.get('nodes').get(vertex.get('labels').get('{0}')[0]).get('value') ".format(condition_part1) + " " + str(desired_operator) + " {0} ".format(search_value)     
                    rewritten_condition = "("+rewritten_condition + " or " + "len(set([str(vertex.get('nodes').get(item).get('value')) for item in vertex.get('labels').get('{0}')]).intersection(set([{1}]))) == 0)".format(condition_part1, search_value)
                else:
                    search_value = " '{0}' ".format(condition_part2)
                    rewritten_condition = "vertex.get('nodes').get(vertex.get('labels').get('{0}')[0]).get('value') ".format(condition_part1) + " " + str(desired_operator) + " {0} ".format(search_value)     
                    rewritten_condition = "(("+rewritten_condition + ") or " + "(len(set([str(vertex.get('nodes').get(item).get('value')) for item in vertex.get('labels').get('{0}')]).intersection(set([{1}]))) != 0))".format(condition_part1, search_value)

        else: # comparison to JSON doc value
            if condition_part1 == condition_part2:
                rewritten_condition = "len([vertex.get('nodes').get(item).get('value') for item in vertex.get('labels').get('{0}')]) != len(set([vertex.get('nodes').get(item).get('value') for item in vertex.get('labels').get('{0}')]))".format(condition_part1, condition_part2)
            else:
                rewritten_condition = "vertex.get('nodes').get(vertex.get('labels').get('{0}')[0]).get('value') ".format(condition_part1) + " " + str(desired_operator) + "vertex.get('nodes').get(vertex.get('labels').get('{0}')[0]).get('value') ".format(condition_part2)     
                rewritten_condition = "("+rewritten_condition + " or "+"len(set([vertex.get('nodes').get(item).get('value') for item in vertex.get('labels').get('{0}')]).intersection(set([vertex.get('nodes').get(item).get('value') for item in vertex.get('labels').get('{1}')]))) != 0)".format(condition_part1, condition_part2)

        return rewritten_condition
        
    split_conditions = []
    variable_names = []
    readable_condition = ""
    
    # Retrieve the conditions and logical operators
    try: 
        logical_operators = re.findall(' and not | or not | and | or | not ', conditions) + [" "]
    except TypeError:
        logical_operators = [" "]

    conditions = re.split(' and not | or not | and | or | not ', conditions)
    
    # Write or conditions to machine executable syntax
    or_status = False
    or_counter = 0
    for i in range(0, len(conditions)):
        condition = conditions[i]
        translated_condition = regex_condition(condition)
        l_o = logical_operators[i]
        if l_o in ["or", "or not", " or ", " or not ", "or ", "or not"]:
            readable_condition = readable_condition + "("+translated_condition + logical_operators[i]  
            or_status = True
            or_counter += 1
        else:
            if l_o == " ":
                if or_status is True:
                    if or_counter != 1:
                        readable_condition = readable_condition + translated_condition+ "))"
                    else:
                        readable_condition = readable_condition + translated_condition+ ")"
                    or_status = False
                    or_counter = 0
                else:
                    readable_condition = readable_condition + translated_condition 
            else:
                if or_status is True:
                    readable_condition = readable_condition + translated_condition+ ")" + logical_operators[i]  
                    or_status = False
                    or_counter = 0
                else:
                    readable_condition = readable_condition + translated_condition + logical_operators[i] 
            
    # input collection
    if type(C[0]) == str:
        C = C[0]
        C = eval(C)

    output = []
    # perform selection
    for vertex in C:
        try:
            if eval(readable_condition, {"vertex": vertex}) is True:
                output.append(vertex)
        except: TypeError
            #print("TypeError selection")
    
    return output


**Unorder operator**

In [10]:
def unorder(C):
    """ This function makes an ordered collection unordered"""
    random.seed(123)
    random.shuffle(C)
    return C

**Distinct**

In [11]:
def distinct(C, unique_identifier): 
    """ This function removes all duplicates from a collection based on some unique identifier key"""
    found_identifiers = [None]
    distinct_collection = []
    
    for tree in C:
        temp_identifier = None
        
        try:
            temp_identifier = tree.get("nodes").get(tree.get("labels").get(unique_identifier)[0]).get('value')
        except:
            None
        
        if temp_identifier not in found_identifiers:
            found_identifiers.append(temp_identifier)
            distinct_collection.append(tree)
            
        
    return distinct_collection

**Sort**

In [12]:
def sort(col, labels): 
    """ This function sorts a collection based on some labels, where the first label determines the first round of ordering"""
    labels.reverse()
    
    for label in labels:
        if "descending" in label: # collection should be sorted descendingly based on the label
            variable_name = label.strip("descending").replace("$", "").lstrip().rstrip().replace("'", "")
            col.sort(key = lambda x: x.get("nodes").get(x.get("labels").get(variable_name)[0])['value'], reverse=True)
        else: # collection should be sorted ascendingly based on the label
            variable_name = label.strip("ascending").replace("$", "").lstrip().rstrip().replace("'", "")
            col.sort(key = lambda x: x.get("nodes").get(x.get("labels").get(variable_name)[0])['value'], reverse=False)
    
    return col

**Join** 

In [13]:
def retrieve_value(doc_in_tree, name):
    """ This function retrieve a specfic node value, this function is utilized in the join function """
    identifier = doc_in_tree.get("labels").get(name.strip("'"))
    
    try: 
        if len(identifier) != 1:
            result = []
            for temp_iden in identifier:
                result.append(doc_in_tree.get("nodes").get(temp_iden).get("value"))
            return set(result) # als het goed is is deze unordered
        else:
            return doc_in_tree.get("nodes").get(identifier[0]).get("value")
    except None or TypeError:
        #print("AN ERROR OCCURED, NODE COULD NOT BE FOUND!")
        return None

In [14]:
def join(collection_names, conditions):
    """ This function joins two collections based on some conditions"""
    # input collections 
    if type(collection_names[0]) == str:
        c1 = eval(collection_names[0])
    else:
        c1 = collection_names[0]
        
    if type(collection_names[1]) == str:
        c2 = eval(collection_names[1])
    else:
        c2 = collection_names[1]
    
    # transform condition(s) if provided
    condition = conditions 
    desired_operator = re.findall(r'>=|=<|=|<|>', condition)[0]
    variable_names = re.split(r'>=|=<|=|<|>', condition)
    
    if collection_names[0] in variable_names[0]:
        condition_part1 = re.split(r'\$', variable_names[0])[-1].strip("'").strip('"').rstrip().lstrip().strip("`").strip("'")
        condition_part2 = re.split(r'\$', variable_names[1])[-1].strip("'").strip('"').rstrip().lstrip().strip("`").strip("'")
    else:
        condition_part2 = re.split(r'\$', variable_names[0])[-1].strip("'").strip('"').rstrip().lstrip().strip("`").strip("'")
        condition_part1 = re.split(r'\$', variable_names[1])[-1].strip("'").strip('"').rstrip().lstrip().strip("`").strip("'")
    
    join_result = []

    for c1_doc in c1:
        retrieved_value_c1 = retrieve_value(c1_doc, condition_part1) 
        
        for c2_doc in c2:
            retrieved_value_c2 = retrieve_value(c2_doc, condition_part2)
            d_o = desired_operator.strip(" ")
            if d_o == "=": # python syntax 
                if retrieved_value_c1 == retrieved_value_c2: 
                    # add the join into the new result
                    # it now basically is a cartesian prodcut of tow singleton collections
                    join_result.append(cartesian_product([[c1_doc], [c2_doc]])[0])
            else:
                if eval(retrieved_value_c1 +" " + d_o + " " + retrieved_value_c2):
                    join_result.append(cartesian_product([[c1_doc], [c2_doc]])[0])
                
    return join_result

### **Cartesian Product**

In [15]:
def cartesian_product(C): 
    """ This function performs a cartesian product over de collections provided in C"""
    # input collections
    if type(C[0]) == str:
        C1 = eval(C[0])
    else:
        C1 = C[0]
    if type(C[1]) ==  str:
        C2 = eval(C[1])
    else:
        C2 = C[1]
    
    cartesian_result = []

    for tree_from_c1 in C1:
        current_tree_from_c1 = tree_from_c1.copy()  #always .copy() when iterating over dictionaries
        num_nodes = len(current_tree_from_c1.get('nodes').keys())
        num_edges = len(current_tree_from_c1.get('edges').keys())
        
        nodes_tree1 = current_tree_from_c1.get("nodes")
        edges_tree1 = current_tree_from_c1.get("edges")
        labels_tree1 = current_tree_from_c1.get("labels")
        
        # add the second tree to the cartesian product, while updating the identifiers for the nodes and edges (+ their referals)
        for tree_from_c2 in C2:
            current_tree_from_c2 = tree_from_c2.copy()
            
            current_nodes = nodes_tree1.copy()
            current_edges = edges_tree1.copy()
            current_labels = labels_tree1.copy()
            
            for node in list(current_tree_from_c2.get("nodes").keys()):
                specific_node_t2 = current_tree_from_c2.get("nodes").get(node).copy()
                
                if specific_node_t2.get('type') == 'complex':
                    specific_node_t2['value'] = specific_node_t2['value'] + num_nodes
                
                specific_node_t2["outgoingEdge"] = [e+num_edges for e in specific_node_t2["outgoingEdge"]]
    
                current_nodes[node+num_nodes] = specific_node_t2
            
            for edge in list(current_tree_from_c2.get("edges").keys()):
                specific_edge_t2 = current_tree_from_c2.get("edges").get(edge).copy()
                
                specific_edge_t2['parent'] = int(specific_edge_t2["parent"]) + num_nodes
                specific_edge_t2['child'] = int(specific_edge_t2["child"]) + num_nodes
                    
                current_edges[edge+num_edges] = specific_edge_t2
            
            for l in list(current_tree_from_c2.get("labels").keys()):
                
                temp_list = []
                for child_reference in current_tree_from_c2.get("labels").get(l):
                    temp_list.append(child_reference+num_nodes)

                if l in list(current_labels.keys()):
                    current_labels[l] = current_labels.get(l) + temp_list
                else:
                    current_labels[l] = temp_list
                
            tree_product = {
                'nodes': current_nodes,
                'edges': current_edges,
                'labels': current_labels
            }
            
            cartesian_result.append(tree_product)
            
    return cartesian_result
           

**Union**

In [16]:
def union(C1, C2):
    """ This function adds two collections together """
    result = C1 + C2
    return result

**Intersection**

In [17]:
def intersection(C1, C2, unique_identifier):
    """ This function returns the JSON documents that are present in both collection, note that in order to compare if two JSON documents are identical a unique identifer key should be specified"""
    found_identifiers = [None]
    result = []
    
    for tree in C2:
        temp_identifier = None
        
        try:
            temp_identifier = tree.get("nodes").get(tree.get("labels").get(unique_identifier)[0]).get('value')
        except:
            None
        
        if temp_identifier is not None:
            found_identifiers.append(temp_identifier)
    
    for tree in C1:
        temp_identifier = None
        
        try:
            temp_identifier = tree.get("nodes").get(tree.get("labels").get(unique_identifier)[0]).get('value')
        except:
            None
        
        if temp_identifier is not None and temp_identifier in found_identifiers: # an intersecting document has been found
            result.append(tree)
    
    return result

**Difference**

In [18]:
def difference(C1, C2, unique_identifier):
    """ This function returns all JSON documents that are present in the first collection but not in the second collection (agian note the unique identifier)"""
    found_identifiers = [None]
    result = []
    
    for tree in C2:
        temp_identifier = None
        
        try:
            temp_identifier = tree.get("nodes").get(tree.get("labels").get(unique_identifier)[0]).get('value')
        except:
            None
        
        if temp_identifier is not None:
            found_identifiers.append(temp_identifier)
    
    for tree in C1:
        temp_identifier = None
        
        try:
            temp_identifier = tree.get("nodes").get(tree.get("labels").get(unique_identifier)[0]).get('value')
        except:
            None
        
        if temp_identifier is not None and temp_identifier not in found_identifiers: # a difference has been found
            result.append(tree)
    
    return result
            

Specifiying the algebraic operators done!

## JSONiq 

JSONiq utilizes the engine RUMBLE (source: https://colab.research.google.com/github/RumbleDB/rumble/blob/master/RumbleSandbox.ipynb#scrollTo=oKidcQCmLsS5)

In [19]:
import requests
import json
import time
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def rumble(line, cell=None):
    if cell is None:
        data = line
    else:
        data = cell

    start = time.time()                                                         
    response = json.loads(requests.post(server, data=data).text)                   
    end = time.time()                                                              
    print("Took: %s ms" % (end - start))

    if 'warning' in response:
        print(json.dumps(response['warning']))
    if 'values' in response:
        for e in response['values']:
            print(json.dumps(e))
    elif 'error-message' in response:
        return response['error-message']
    else:
        return response

In [20]:
# we run jsoniq on a public server for convenience
server ='http://public.rumbledb.org:9090/jsoniq'

##  JSONiq query to JSON Algebra

**Lexer + Parser**

In [21]:
# Based on : https://gist.github.com/eliben/5797351
class Token(object): 
    """ A simple Token structure. Contains the token type, value and position. """
    def __init__(self, type, val, pos):
        self.type = type
        self.val = val
        self.pos = pos

    def __str__(self):
        return '%s(%s) at %s' % (self.type, self.val, self.pos)


class LexerError(Exception):
    """ Throw lexer error """
    def __init__(self, pos):
        self.pos = pos


class Lexer(object):
    """ A regex-based lexer/tokenizer """
    def __init__(self, rules, skip_whitespace=True):
        """ initialize a lexer """ 
        idx = 1
        regex_parts = []
        self.group_type = {}

        for regex, type in rules:
            groupname = 'GROUP%s' % idx
            regex_parts.append('(?P<%s>%s)' % (groupname, regex))
            self.group_type[groupname] = type
            idx += 1

        self.regex = re.compile('|'.join(regex_parts))
        self.skip_whitespace = skip_whitespace
        self.re_ws_skip = re.compile('\S')

    def input(self, q):
        """ Initialize the lexer """
        self.query = q
        self.pos = 0

    def token(self):
        """ Return the next token (a Token object) found in the input """
        if self.pos >= len(self.query):
            return None
        else:
            if self.skip_whitespace:
                m = self.re_ws_skip.search(self.query, self.pos)

                if m:
                    self.pos = m.start()
                else:
                    return None

            m = self.regex.match(self.query, self.pos)
            if m:
                groupname = m.lastgroup
                tok_type = self.group_type[groupname]
                tok = Token(tok_type, m.group(groupname), self.pos)
                self.pos = m.end()
                return tok

            # if here, no rule was matched
            raise LexerError(self.pos)

    def tokens(self):
        """ Returns an iterator to the tokens found in the input """
        while 1:
            tok = self.token()
            if tok is None: break
            yield tok


In [22]:
def initialize_lexer():
    """ This function initalizes the required lexer"""
    
    # Necessary lexer rules for the translation to JAL
    rules = [
            ('let \$.+(:=)','VARIABELE ASSIGNMENT'),
            
            ("for \$.+? in .+?(?=for|where|return|for|let|order by)", 'PROJECTION'),
            ("where \$.*?(?=for|return|for|let|order by|where)", 'CONDITION'),
            
        
            ('return .+?(?=return)', 'RETURN'),
            ('return .+', 'RETURN'),
        
            ('order by .*?(?=return|for|let|order by)', "SORT")
        ]

    lx = Lexer(rules, skip_whitespace=True)
    return lx


In [23]:
#  create the lexer
lx = initialize_lexer()

In [24]:
def parse_lexer_part1(q): 
    """ This function parses the query to a basic syntax, that later (part2) will be used for the tranlation to actual JAL"""
    
    # Lexer
    lx.input(q)
    lex_result = []
    try:
        for match in lx.tokens():
            lex_result.append(str(match).split(r" at ")[0])
    except LexerError as err:
        print('LexerError at position %s' % err.pos)
    
    # tranlate the lexer result to a query process
    query_process = []
    function_call = []
    while lex_result != []:
        action = lex_result[0]
        

        if len(lex_result) >1:
            
            if function_call == []:
                # start of a new clause, meaning that it has to begin with either let or for
                if "VARIABELE ASSIGNMENT" in action: # skip 
                    lex_result = lex_result[1:]

                if "PROJECTION" in action:
                    function_call.append(action)
                    lex_result = lex_result[1:]
                    
            else: 
                if "PROJECTION" in action or "CONDITION" in action:
                    function_call.append(action)
                    lex_result = lex_result[1:]
            
                if "VARIABELE ASSIGNMENT" in action: # skip 
                    lex_result = lex_result[1:]
                
                if "SORT" in action:
                    function_call.append(action)
                    lex_result = lex_result[1:]

                if "RETURN" in action:
                    function_call.append(action)
                    query_process.append(function_call)
                    function_call = []
                    lex_result = lex_result[1:]
            
        else:
            # has to be a return statement
            if not "RETURN" in action:
                return "Error, the query does not end with a return clause!"
            else:
                # Assume that it is the return clause belonging to the let clause (so does not have to be added to query process)
                lex_result = []
    
    return query_process

In [25]:
def algebra_input(converted_query): # second part for the parsing
    """ This function tranlates the query process result from parser part 1 (previous function) to JAL"""
    
    operator_calls = []
    
    for clause in converted_query:
        call = []
        condition_clause_temp_variables = []
        collection_names = []
        conditions_total = ""
        for action in clause:
            if "PROJECTION" in action:
                # find collection name(s)
                temp_collection_names = re.findall(r"\$([a-zA-Z]+)\[\]", action)
                collection_names = collection_names + temp_collection_names
                
                temp_names = re.findall(r"\$([a-zA-Z]+) in", action)
                for i in range(0, len(temp_names)):
                    condition_clause_temp_variables.append([temp_names[i], temp_collection_names[i]])
                
                
            if "CONDITION" in action:
                conditions = action.strip("CONDITION(where").strip(")").lstrip().rstrip().strip("'").replace(")", "").replace("jn:members(", "")

                for temp_key in condition_clause_temp_variables:
                    conditions = conditions.replace(r"${0}.".format(temp_key[0]), "${0}.".format(temp_key[1]))
                
                if len(conditions_total) == 0:
                    conditions_total = conditions
                else:
                    conditions_total = conditions_total + " and " + conditions
            
            if "SORT" in action:
                if len(call) == 0: # order by clause over a stand-alone projection
                    if len(collection_names) == 1:
                        if len(conditions_total) != 0:
                            call.append("selection")
                            call.append(collection_names)
                            call.append(conditions_total)
                        else:
                            call.append("projection")
                            call.append(collection_names)
                    else:
                        call.append("join") 
                        call.append(collection_names)
                        call.append(conditions_total)
                        
                
                sort_labels = action.strip("SORT(order by").strip(")").lstrip().rstrip().split(",")
                
                for temp_key in condition_clause_temp_variables:
                    for k in range(0, len(sort_labels)):
                        sort_labels[k] = sort_labels[k].replace(r"${0}.".format(temp_key[0]), r"${0}.".format(temp_key[1]))
                    
                call.append("sort")
                call.append(sort_labels)
            
            
            if "RETURN" in action:
                
                if call == []:
                    if len(collection_names) == 1:
                        if conditions_total != "":
                            call.append("selection")
                            call.append(collection_names)
                            call.append(conditions_total)
                    else:
                        call.append("join") 
                        call.append(collection_names)
                        call.append(conditions_total)
                
                if bool(re.match(r'RETURN\(return \{.+\}.+', action)) is True: # white spaces are a bottleneck here
                    labels = re.split(",", action)
                    for temp_key in condition_clause_temp_variables:
                        for j in range(0, len(labels)):
                            temp_label_j = labels[j].replace("{0}.".format(temp_key[0]), "{0}.".format(temp_key[1])).replace("'", "").replace('"', "").replace(")", "").replace("}", "").replace("$", "").rstrip()
                            labels[j] = temp_label_j.split(":")[-1].lstrip().rstrip()
                    call.append("projection")
                    call.append(labels)  
                
                if bool(re.match(r'RETURN\(return \$.+', action)) is True: 
                    # intermediate output is the output   
                    pass
                    
        operator_calls.append(call)        
    
    return operator_calls

In [26]:
def tree_to_collection(input_collection_trees):
    """ Convert a collection of trees back to a collection of JSON documents """
    
    output = []
    
    
    def find_value(tree_info, n, e_label, traversed_edge_identifiers):
        """ this function find the value of the node the edge refers to"""
        if n.get("type") == "atomic": # node is of atomic type, hence a leaf node and further traversal is not required
            node_value = n.get("value")
            if len(traversed_edge_identifiers) == 1:
                e_label = e_label.split(".")[-1]
                return {e_label: node_value}, None
            else: 
                return node_value, None
        
        else: # node is complex, furter traversal is required
            type_n_edge = tree_info.get("edges").get(n.get("outgoingEdge")[0]).get("type")
                        
            if type_n_edge == "int": # then the complex type refers to a list
                temp_list = []
                for e in n.get("outgoingEdge"):
                    e_info = tree_info.get("edges").get(e)
                    traversed_edge_identifiers.append(e)
                    temp_list.append(find_value(tree_info, tree_info.get("nodes").get(e_info.get("child")), e_info.get("label"), traversed_edge_identifiers)[0])
                e_label = e_label.split(".")[-1]
                return {e_label: temp_list}, traversed_edge_identifiers
            else: # complex type refers to a dictionary
                temp_dict = {}
                for e in n.get("outgoingEdge"):
                    try:
                        e_info = tree_info.get("edges").get(e)
                        traversed_edge_identifiers.append(e)
                        temp_dict[e_info.get("label")] = find_value(tree_info, tree_info.get("nodes").get(e_info.get("child")), e_info.get("label"), traversed_edge_identifiers)[0]
                    except: IndexError
                e_label = e_label.split(".")[-1]
                return {e_label: temp_dict}, traversed_edge_identifiers
                
    
    # Convert each tree from the collection to a document
    for doc in input_collection_trees:
        doc_edges = list(doc.get("edges").keys())
        doc_result = { }

        while doc_edges != []:
            e_info = doc.get("edges").get(doc_edges[0])
            if e_info.get("type") == "str":
                node = doc.get("nodes").get(doc.get("labels").get(e_info.get("path"))[0])
            else:
                node = doc.get("nodes").get(e_info.get("child"))
            traversed_identifiers = [doc_edges[0]]
            temp_value, tr_id = find_value(doc, node, e_info.get("label"), traversed_identifiers)
            doc_result[doc_edges[0]] = temp_value
            traversed_identifiers.append(tr_id)
            doc_edges = [e for e in doc_edges if e not in traversed_identifiers]

        # add document to output
        temp_output = { }
        for temp_res in doc_result.values():
            temp_output.update(temp_res)
        output.append(temp_output)
        
    return output

In [27]:
def optimizer(unoptimzed_order):
    """ this function applies my optimization strategy to the input query expressed in JAL """
    
    process_order = []
    sorted_possibly = []
    
    for process in unoptimzed_order:
        temp_order = []
        sub_process = process
        
        while sub_process != []:
            function_name = sub_process[0]

            if function_name == "join" or function_name == "selection":
                # decompse the join if possible
                # collection sizes
                relevant_collections = process[1]
                collection_sizes = {}
                for collection in relevant_collections:
                    collection_sizes[collection] = len(eval(collection))

                collection_sizes = sorted(collection_sizes.items(), key=lambda x: x[1])
                
                # perform most restrictive selections (= conditions) first
                conditions = process[2]
                and_conditions = re.split(' and ', conditions) 
                most_restrictive_conditions = []
                inbetween_conditions = []
                least_favourable_conditons = []
                for and_condition in and_conditions:
                    logics = re.findall(r'and not|or not| or | and ', and_condition)
                    desired_operator = re.findall(r'>=|<=|=|<|>', and_condition)
                    variable_names = re.split(r'>=|<=|=|<|>', and_condition)
                    
                    if len(variable_names) <= 2:
                        desired_operator = desired_operator[0]
                        if desired_operator == "=": 
                            if not "$" in variable_names[1]:
                                if "not $" in variable_names[0]:
                                    most_restrictive_conditions.append(variable_names[0].replace("not", "") + "!=" + variable_names[1])
                                else:
                                    most_restrictive_conditions.append(variable_names[0] + desired_operator + variable_names[1])
                            else:
                                if "not $" in variable_names[0]:
                                    least_favourable_conditons.append(variable_names[1] + "!=" + variable_names[0].replace("not", ""))
                                else:
                                    least_favourable_conditons.append(variable_names[0] + desired_operator + variable_names[1])
                        else:
                            if not "$" in variable_names[1]:
                                if "not $" in variable_names:
                                    inbetween_conditions.append(variable_names[1] + desired_operator + variable_names[0])
                                else:
                                    inbetween_conditions.append(variable_names[0] + desired_operator + variable_names[1])
                            else:
                                if "not $" in variable_names:
                                    least_favourable_conditons.append(variable_names[1] + desired_operator + variable_names[0])
                                else:
                                    least_favourable_conditons.append(variable_names[0] + desired_operator + variable_names[1])
                    else:
                        temp_most_restrictive = []
                        temp_inbetween_restrictive = []
                        temp_least_restrictive = []
                        logics = logics + [" "]
                        temp_operators = desired_operator
                        variable_names = re.split(r' and | or ', and_condition)
                        temp_total = []
                        for temp_vn in variable_names:
                            temp_total = temp_total + re.split(r'>=|=<|=|<|>', temp_vn)
                        variable_names = temp_total
                        for t in range(0, int(len(variable_names)/2)):
                            temp_logic = logics[t]
                            desired_operator = temp_operators[t]
    
                            if desired_operator == "=": 
                                if not "$" in variable_names[t*2+1]:
                                    if " not $" in variable_names[2*t]:
                                        temp_most_restrictive.append(variable_names[2*t].replace("not", "") + "!=" + variable_names[t*2+1] + " " + temp_logic)
                                    else:
                                        temp_most_restrictive.append(variable_names[2*t] + desired_operator + variable_names[t*2+1] + " " + temp_logic)
                                else:
                                    if " not $" in variable_names[t*2+1]:
                                        temp_least_restrictive.append(variable_names[t*2+1] + "!=" + variable_names[t].replace("not", "") + " " + temp_logic)
                                    else:
                                        temp_least_restrictive.append(variable_names[2*t] + desired_operator + variable_names[t*2+1] + " " + temp_logic)
                            else:
                                if not "$" in variable_names[t*2+1]:
                                    if "not $" in variable_names:
                                        temp_inbetween_restrictive.append(variable_names[t*2+1] + desired_operator + variable_names[2*t] + " " + temp_logic)
                                    else:
                                        temp_inbetween_restrictive.append(variable_names[2*t] + desired_operator + variable_names[t*2+1] + " " + temp_logic)
                                else:
                                    if "not $" in variable_names:
                                        temp_least_restrictive.append(variable_names[t*2+1] + desired_operator + variable_names[2*t] + " " + temp_logic)
                                    else:
                                        temp_least_restrictive.append(variable_names[2*t] + desired_operator + variable_names[t*2+1] + " " + temp_logic)
                        
                        if temp_inbetween_restrictive == [] and temp_least_restrictive == []:
                            concantenated_condition = ""
                            for entry in temp_most_restrictive:
                                concantenated_condition = concantenated_condition + " " + entry
                            most_restrictive_conditions.append(concantenated_condition)
                        elif temp_least_restrictive == []:
                            concantenated_condition = ""
                            for entry in temp_most_restrictive:
                                concantenated_condition = concantenated_condition + " " + entry
                            for entry in temp_inbetween_restrictive:
                                concantenated_condition = concantenated_condition + " " + entry
                            inbetween_conditions.append(concantenated_condition)
                        else:
                            concantenated_condition = ""
                            for entry in temp_most_restrictive:
                                concantenated_condition = concantenated_condition + " " + entry
                            for entry in temp_inbetween_restrictive:
                                concantenated_condition = concantenated_condition + " " + entry
                            for entry in temp_least_restrictive:
                                concantenated_condition = concantenated_condition + " " + entry
                            least_favourable_conditons.append(concantenated_condition)
                            

                while most_restrictive_conditions != []:
                    for smallest_collection in collection_sizes:
                        smallest_collection = smallest_collection[0]
                        for most_restrictive_condition in most_restrictive_conditions:
                            if smallest_collection in most_restrictive_condition:
                                temp_order.append(['selection', [smallest_collection], most_restrictive_condition])
                                most_restrictive_conditions.remove(most_restrictive_condition)

                while inbetween_conditions != []:
                    for smallest_collection in collection_sizes:
                        smallest_collection = smallest_collection[0]
                        for inbetween_condition in inbetween_conditions:
                            if smallest_collection in inbetween_condition:
                                temp_order.append(['selection', [smallest_collection], inbetween_condition])
                                inbetween_conditions.remove(inbetween_condition)
                
                ordered_collections = []
                str_ordered_collections = ""
                for colle in collection_sizes:
                    ordered_collections.append(colle[0])
                    str_ordered_collections = str_ordered_collections + str(colle[0]) + "|"
                str_ordered_collections = str_ordered_collections[:-1]
                
                while least_favourable_conditons != []:
                    for smallest_collection in ordered_collections:
                        join_condition_order = ""
                        for least_favourable_conditon in least_favourable_conditons:
                            if smallest_collection in least_favourable_conditon:
                                join_collections = re.findall(r"{0}".format(str_ordered_collections), least_favourable_conditon)
                                join_condition_order = join_condition_order + least_favourable_conditon
                                least_favourable_conditons.remove(least_favourable_conditon)
                                temp_order.append(['join', list(set(join_collections)), join_condition_order])
                sub_process = sub_process[3:]
            
            if function_name == "projection":
                temp_order.append(['projection', sub_process[1]])
                sub_process = sub_process[2:]
            
            if function_name == "sort":
                sorted_possibly = sorted_possibly + ['sort', sub_process[1]]
                sub_process = sub_process[2:]
                
        process_order.append(temp_order)
    process_order[0].append(sorted_possibly)
    return process_order

In [28]:
def execution(query, optimization): 
    """ This function executes a query with optimization """
    start = time.time() 
    
    # Parsing
    part1 = parse_lexer_part1(query)
    process_order = algebra_input(part1)

    if optimization is True:
        process_order = optimizer(process_order)[0]
    query_result = None

    updated_collections = {} # to keep track of intermediate collection results
    joined_collections = [] # to keep track of the collections that are already present in a join
    
    for q in process_order:
        temp = q
        temp_collection = []
        
        while temp != []:
            function_name = temp[0]

            if function_name == 'join':
                collection_names = temp[1]
                                
                if query_result is None:
                    first_col = collection_names[0]
                    joined_collections.append(first_col)
                    if first_col in updated_collections.keys(): # an operation has already been performed on the collection
                        first_col = updated_collections.get(first_col)
                        
                    query_result = first_col
                    for i in range(1, len(collection_names)):
                        next_col = collection_names[i]
                        
                        if next_col not in joined_collections:
                            joined_collections.append(next_col)
                            if next_col in updated_collections.keys(): # an operation has already been performed on the collection
                                next_col = updated_collections.get(next_col)
        
                            query_result = cartesian_product([query_result, next_col])

                    
                    if query_result == first_col: # technically there is no query result yet
                        query_result = None
  
                else: # an intermediate query result already exists 
                    for i in range(0, len(collection_names)):
                        collection_name_name = collection_names[i]
                    
                        if collection_name_name in updated_collections.keys(): # an operation has already been performed on the collection
                            collection_name = updated_collections.get(collection_name_name)
                            
                        if collection_name_name not in joined_collections: # collection is not yet present in the join 
                            if type(collection_name_name) == str: 
                                collection_name = collection_name_name
                            query_result = cartesian_product([query_result, collection_name])
                            joined_collections.append(collection_name_name)

                if temp[2] != "": # a selection should be performed
                    query_result = selection(query_result, temp[2])
                temp = temp[3:]

            elif function_name == "projection":
                if optimization is True:
                    if query_result == None:
                        temp_result = updated_collections.get(list(updated_collections.keys())[0])
                        query_result = projection(temp_result, temp[1])
                        temp_collection.append(temp[1])
                    else:
                        query_result = projection(query_result, temp[1])
                    temp = temp[3:]
                else:
                    if query_result == None:
                        temp_collection.append(temp[1])
                    else:
                        query_result = projection(query_result, temp[1])
                    temp = temp[2:]
                
            elif function_name == "selection":
                if optimization is True:
                    if temp[1][0] in updated_collections.keys():
                        updated_collections[temp[1][0]] = selection(updated_collections.get(temp[1][0]), str(temp[2]))
                    else:
                        updated_collections[temp[1][0]] = selection(temp[1], str(temp[2]))
                else:
                    query_result = selection(temp[1], temp[2])
                temp =  temp[3:]
            
            elif function_name == "sort":
                if query_result is None: # the query does not do any selections, or joins
                    query_result = temp_collection[0][0]
                else:
                    query_result = sort(query_result, temp[1])
                temp = temp[2:]
            
            else:
                return None
                
    
    end = time.time()   
    print("Took: %s ms" % (end - start))
    return tree_to_collection(query_result)

In [29]:
def execution_without_optimization(query): 
    """ This function executes a query without optimization """
    start = time.time() 
    
    # Parser 
    part1 = parse_lexer_part1(query)
    part2 = algebra_input(part1)

    process_order = part2
    
    query_result = None
    
    for q in process_order:
        temp = q
        temp_collection = []
        while temp != []:
            function_name = temp[0]

            if function_name == 'join':
                collection_names = temp[1]
                if query_result is None:
                    query_result = collection_names[0]
                    for i in range(1, len(collection_names)):
                        query_result = cartesian_product([query_result, collection_names[i]])
                else:
                    for i in range(0, len(collection_names)):
                        query_result = cartesian_product([query_result, collection_names[i]])
                
                if temp[2] != "":
                    full_condition = temp[2]
                    if " or " in full_condition:
                        full_condition = full_condition.split("and")
                        total = []
                        temp_c = ""
                        for temp_fc in full_condition:
                            if " or " in temp_fc:
                                total.append(temp_c.rstrip(" and "))
                                total.append(temp_fc)
                                temp_c = ""
                            else: 
                                temp_c = temp_c + temp_fc + " and "
                        for condition_split in total:
                            query_result = selection(query_result, condition_split)
                    else:
                        query_result = selection(query_result, temp[2])
                temp = temp[3:]

            elif function_name == "projection":
                if query_result == None:
                    temp_collection.append(temp[1])
                    temp = temp[2:]
                else:
                    query_result = projection(query_result, temp[1])
                    temp = temp[2:]
                
            elif function_name == "selection":
                query_result = selection(temp[1], temp[2])
                temp =  temp[3:]
            
            elif function_name == "sort":
                if query_result is None:
                    query_result = temp_collection[0][0]
                    query_result = sort(eval(query_result), temp[1])
                else:
                    query_result = sort(query_result, temp[1])
                temp = temp[2:]
            
            else:
                return None
                
    
    end = time.time()   
    print("Took: %s ms" % (end - start))
    return tree_to_collection(query_result)

# Examples

### **Database 1: Pokemons**

**Query 1**: *Retrieve all pokemon names on alphabetical order that weight 9.5 kg and have weakness Ground and/or Psychic*

- Parse query in JSONiq

In [30]:
%%rumble 
let $pokemon := [
    ]
let $result := for $pokemon in $pokemon[] 
    where $pokemon."weight" = "9.5 kg" 
    and jn:members($pokemon."weaknesses") = "Ground" or jn:members($pokemon."weaknesses") = "Psychic" 
            order by $pokemon."name" ascending 
            return {"name": $pokemon."name"} 
        return [$result]

Took: 0.12381625175476074 ms
[]


- Query 1 without optimization

In [31]:
# load database 
f = open('Data/Pokemons.json', )
data = json.load(f).get("pokemon")
pokemon = json_datamodel(data, "pokemon")
f.close()

query_1 = "let $result := for $pokemon in $pokemon[] where $pokemon.'weight' = '9.5 kg' and jn:members($pokemon.'weaknesses') = 'Ground' or jn:members($pokemon.'weaknesses') = 'Psychic' order by $pokemon.'name' ascending return {'name': $pokemon.'name'} return [$result]"
result_1 = execution_without_optimization(query_1)
print("OUTPUT QUERY 1 WITHOUT OPTIMIZATION: ", result_1)
print("LENGTH OUTPUT QUERY 1 WITHOUT OPTIMIZATION: ", len(result_1))

Took: 0.028985023498535156 ms
OUTPUT QUERY 1 WITHOUT OPTIMIZATION:  [{'name': 'Weezing'}]
LENGTH OUTPUT QUERY 1 WITHOUT OPTIMIZATION:  1


- Query 1 with optimization

In [123]:
f = open('Data/Pokemons.json', )
data = json.load(f).get("pokemon")
pokemon = json_datamodel(data, "pokemon")
f.close()

query_1 = "let $result := for $pokemon in $pokemon[] where $pokemon.'weight' = '9.5 kg' and jn:members($pokemon.'weaknesses') = 'Ground' or jn:members(pokemon.'weaknesses') = 'Psychic' order by $pokemon.'name' ascending return {'name': $pokemon.'name'} return [$result]"
result_1 = execution(query_1, True)
print("OUTPUT QUERY 1 WITH OPTIMIZATION: ", result_1)
print("LENGTH OUTPUT QUERY 1 WITH OPTIMIZATION: ", len(result_1))

Took: 0.010460853576660156 ms
OUTPUT QUERY 1 WITH OPTIMIZATION:  [{'name': 'Weezing'}]
LENGTH OUTPUT QUERY 1 WITH OPTIMIZATION:  1


**Query 2**: *Retrieve all Pokemon ids in ascending order that weigh 2.5 kg or more, have height 0.5 m or less and have weakness Electric and/or Flying*

- Parse query in JSONiq

In [124]:
%%rumble 
let $pokemon := []
let $result := 
    for $pokemon in $pokemon[] where $pokemon."weight" >= "2.5 kg" and $pokemon."height" <= "0.5 m"
    and jn:members($pokemon."weaknesses") = "Electric" or jn:members($pokemon."weaknesses") = "Flying"
    order by $pokemon."id" ascending return {"id": $pokemon."id"} 
return [$result]

Took: 0.770050048828125 ms
[]


- Query 2 without optimization

In [125]:
f = open('Data/Pokemons.json', )
data = json.load(f).get("pokemon")
pokemon = json_datamodel(data, "pokemon")
f.close()

query_2 = "let $result := for $pokemon in $pokemon[] where $pokemon.'weight' >= '2.5 kg' and $pokemon.'height' <= '0.5 m' and  jn:members($pokemon.'weaknesses') = 'Electric' or jn:members($pokemon.'weaknesses') = 'Flying' order by $pokemon.'id' ascending return {'id': $pokemon.'id'} return [$result]"
result_2 = execution_without_optimization(query_2)
print("OUTPUT QUERY 2 WITHOUT OPTIMIZATION: ", result_2)
print("LENGTH OUTPUT QUERY 2 WITHOUT OPTIMIZATION: ", len(result_2))

Took: 0.034896135330200195 ms
OUTPUT QUERY 2 WITHOUT OPTIMIZATION:  [{'id': 10}, {'id': 13}, {'id': 46}, {'id': 90}, {'id': 98}, {'id': 102}, {'id': 116}, {'id': 138}]
LENGTH OUTPUT QUERY 2 WITHOUT OPTIMIZATION:  8


- Query 2 with optimization

In [126]:
f = open('Data/Pokemons.json', )
data = json.load(f).get("pokemon")
pokemon = json_datamodel(data, "pokemon")
f.close()

query_2 = "let $result := for $pokemon in $pokemon[] where $pokemon.'weight' >= '2.5 kg' and $pokemon.'height' <= '0.5 m' and  jn:members($pokemon.'weaknesses') = 'Electric' or jn:members($pokemon.'weaknesses') = 'Flying' order by $pokemon.'id' ascending return {'id': $pokemon.'id'} return [$result]"
result_2 = execution(query_2, True)
print("OUTPUT QUERY 2 WITHOUT OPTIMIZATION: ", result_2)
print("LENGTH OUTPUT QUERY 2 WITHOUT OPTIMIZATION: ", len(result_2))

Took: 0.027441740036010742 ms
OUTPUT QUERY 2 WITHOUT OPTIMIZATION:  [{'id': 10}, {'id': 13}, {'id': 46}, {'id': 90}, {'id': 98}, {'id': 102}, {'id': 116}, {'id': 138}]
LENGTH OUTPUT QUERY 2 WITHOUT OPTIMIZATION:  8


## **2. Airport flight delays** 

**Query 3**: *The retrieval of the airport codes in alphabetical order of all airports that experienced more than 200,000 minutes delay in October of 2003*

- Parse query in JSONiq

In [127]:
%%rumble 
let $airport_month := [ ]
    
let $result := 
    for $airport_month in $airport_month[]
    where $airport_month."Statistics"."Minutes Delayed"."Total" >= 200000 and $airport_month."Time"."Month Name" = "October" and $airport_month."Time"."Year" = 2003
    order by $airport_month."Airport"."Code" ascending
    return {"Code": $airport_month."Airport"."Code"}
return [$result]

Took: 0.3818988800048828 ms
[]


- Query 3 without optimization

In [128]:
f = open('Data/AirportDelays.json', )
data = json.load(f)
airportMonth = json_datamodel(data, "airportMonth")
f.close()

query_3 = "let $result := for $airportMonth in $airportMonth[] where $airportMonth.'Statistics'.'Minutes Delayed'.'Total' >= 200000 and $airportMonth.'Time'.'Month Name' = 'October' and $airportMonth.'Time'.'Year' = 2003 order by $airportMonth.'Airport'.'Code' ascending return {'Code': $airportMonth.'Airport'.'Code'} return [$result]"
result_3 = execution_without_optimization(query_3)
print("OUTPUT QUERY 3 WITHOUT OPTIMIZATION: ", result_3)
print("LENGTH OUTPUT QUERY 3 WITHOUT OPTIMIZATION: ", len(result_3))

Took: 0.40004420280456543 ms
OUTPUT QUERY 3 WITHOUT OPTIMIZATION:  [{'Code': 'ATL'}, {'Code': 'ORD'}]
LENGTH OUTPUT QUERY 3 WITHOUT OPTIMIZATION:  2


- Query 3 with optimization

In [129]:
f = open('Data/AirportDelays.json', )
data = json.load(f)
airportMonth = json_datamodel(data, "airportMonth")
f.close()

query_3 = "let $result := for $airportMonth in $airportMonth[] where $airportMonth.'Statistics'.'Minutes Delayed'.'Total' >= 200000 and $airportMonth.'Time'.'Month Name' = 'October' and $airportMonth.'Time'.'Year' = 2003 order by $airportMonth.'Airport'.'Code' ascending return {'Code': $airportMonth.'Airport'.'Code'} return [$result]"
result_3 = execution(query_3, True)
print("OUTPUT QUERY 3 WITH OPTIMIZATION: ", result_3)
print("LENGTH OUTPUT QUERY 3 WITH OPTIMIZATION: ", len(result_3))

Took: 0.26115918159484863 ms
OUTPUT QUERY 3 WITH OPTIMIZATION:  [{'Code': 'ATL'}, {'Code': 'ORD'}]
LENGTH OUTPUT QUERY 3 WITH OPTIMIZATION:  2


**Query 4**: *The retrieval of the months and years in ascending order, where airport LAX had more than 20 diverted flights while hosting less than 16,000 flights in that month*

- Parse query in JSONiq

In [130]:
%%rumble 
let $airport_month := [ ]
    
let $result := 
    for $airport_month in $airport_month[]
    where $airport_month."Statistics"."Flights"."Total" < 16000 
    and $airport_month."Statistics"."Flights"."Diverted" > 20
    return {"Month Name": $airport_month."Time"."Month", "Year": $airport_month."Time"."Year"}
return [$result]

Took: 0.06972622871398926 ms
[]


- Query 4 without optimization

In [131]:
f = open('Data/AirportDelays.json', )
data = json.load(f)
airportMonth = json_datamodel(data, "airportMonth")
f.close()

query_4 = "let $result := for $airportMonth in $airportMonth[] where $airportMonth.'Airport'.'Code'='LAX' and $airportMonth.'Statistics'.'Flights'.'Total' < 16000 and $airportMonth.'Statistics'.'Flights'.'Diverted' > 20 order by $airportMonth.'Time'.'Year' ascending, $airportMonth.'Time'.'Month Name' ascending return {'Month Name': $airportMonth.'Time'.'Month Name', 'Year': $airportMonth.'Time'.'Year'} return [$result]"
result_4 = execution_without_optimization(query_4)
print("OUTPUT QUERY 4 WITHOUT OPTIMIZATION: ", result_4)
print("LENGTH OUTPUT QUERY 4 WITHOUT OPTIMIZATION: ", len(result_4))

Took: 0.4059009552001953 ms
OUTPUT QUERY 4 WITHOUT OPTIMIZATION:  [{'Month Name': 'November', 'Year': 2009}, {'Month Name': 'Febuary', 'Year': 2010}, {'Month Name': 'November', 'Year': 2010}, {'Month Name': 'Febuary', 'Year': 2011}, {'Month Name': 'Febuary', 'Year': 2015}]
LENGTH OUTPUT QUERY 4 WITHOUT OPTIMIZATION:  5


- Query 4 with optimization

In [133]:
f = open('Data/AirportDelays.json', )
data = json.load(f)
airportMonth = json_datamodel(data, "airportMonth")
f.close()

query_4 = "let $result := for $airportMonth in $airportMonth[] where $airportMonth.'Airport'.'Code'='LAX' and $airportMonth.'Statistics'.'Flights'.'Total' < 16000 and $airportMonth.'Statistics'.'Flights'.'Diverted' > 20 order by $airportMonth.'Time'.'Year' ascending, $airportMonth.'Time'.'Month Name' ascending return {'Month Name': $airportMonth.'Time'.'Month Name', 'Year': $airportMonth.'Time'.'Year'} return [$result]"
result_4 = execution(query_4, True)
print("OUTPUT QUERY 4 WITH OPTIMIZATION: ", result_4)
print("LENGTH OUTPUT QUERY 4 WITH OPTIMIZATION: ", len(result_4))

Took: 0.2571709156036377 ms
OUTPUT QUERY 4 WITH OPTIMIZATION:  [{'Month Name': 'November', 'Year': 2009}, {'Month Name': 'Febuary', 'Year': 2010}, {'Month Name': 'November', 'Year': 2010}, {'Month Name': 'Febuary', 'Year': 2011}, {'Month Name': 'Febuary', 'Year': 2015}]
LENGTH OUTPUT QUERY 4 WITH OPTIMIZATION:  5


## 3. Bike Share Data at Stations

**Query 5**: *The retrieval of the station ids of all stations in the region with id 80 and that have more than 8 bikes available at the station*

- Parse query in JSONiq

In [134]:
%%rumble 
let $station := [ ]
let $status := [ ]
let $result := 
    for $station in $station[], $status in $status[]
    where $station."station_id" = $status."station_id" and $status."num_bikes_available" >= 8 and $station."region_id" = "region_80"
    return {"station_id": $station."station_id"}
return [$result]

Took: 0.07399988174438477 ms
[]


- Query 5 without optimization

In [135]:
f = open('Data/station-information.json', )
data = json.load(f).get('data').get('stations')
station = json_datamodel(data, "station")
f.close()
f = open('Data/station-status.json', )
data = json.load(f).get('data').get('stations')
status = json_datamodel(data, "status")
f.close()

query_5 = "let $result := for $station in $station[], $status in $status[] where $station.'station_id' = $status.'station_id' and $status.'num_bikes_available' >= 8 and $station.'region_id' = 'region_80' return {'station_id': $station.'station_id'} return [$result]"
result_5 = execution_without_optimization(query_5)
print("OUTPUT QUERY 5 WITHOUT OPTIMIZATION: ", result_5)
print("LENGTH OUTPUT QUERY 5 WITHOUT OPTIMIZATION: ", len(result_5))

Took: 1.7056901454925537 ms
OUTPUT QUERY 5 WITHOUT OPTIMIZATION:  [{'station_id': 'hub_290'}, {'station_id': 'hub_299'}, {'station_id': 'hub_314'}, {'station_id': 'hub_632'}, {'station_id': 'hub_1273'}, {'station_id': 'hub_1325'}, {'station_id': 'hub_1782'}, {'station_id': 'hub_2325'}, {'station_id': 'hub_3228'}, {'station_id': 'hub_3229'}, {'station_id': 'hub_5418'}]
LENGTH OUTPUT QUERY 5 WITHOUT OPTIMIZATION:  11


- Query 5 with optimization

In [136]:
f = open('Data/station-information.json', )
data = json.load(f).get('data').get('stations')
station = json_datamodel(data, "station")
f.close()
f = open('Data/station-status.json', )
data = json.load(f).get('data').get('stations')
status = json_datamodel(data, "status")
f.close()

query_5 = "let $result := for $station in $station[], $status in $status[] where $station.'station_id' = $status.'station_id' and $status.'num_bikes_available' >= 8 and $station.'region_id' = 'region_80' return {'station_id': $station.'station_id'} return [$result]"
result_5 = execution(query_5, True)
print("OUTPUT QUERY 5 WITH OPTIMIZATION: ", result_5)
print("LENGTH OUTPUT QUERY 5 WITH OPTIMIZATION: ", len(result_5))

Took: 0.12353992462158203 ms
OUTPUT QUERY 5 WITH OPTIMIZATION:  [{'station_id': 'hub_290'}, {'station_id': 'hub_299'}, {'station_id': 'hub_314'}, {'station_id': 'hub_632'}, {'station_id': 'hub_1273'}, {'station_id': 'hub_1325'}, {'station_id': 'hub_1782'}, {'station_id': 'hub_2325'}, {'station_id': 'hub_3228'}, {'station_id': 'hub_3229'}, {'station_id': 'hub_5418'}]
LENGTH OUTPUT QUERY 5 WITH OPTIMIZATION:  11


**Query 6**: *The retrieval of the station ids and region ids of all stations where the number of available bikes is larger than number of available dockers except for stations in the region 80*

- Parse query in JSONiq

In [137]:
%%rumble 
let $station := [ ]
let $status := [ ]
let $result := for $station in $station[], $status in $status[] 
    where $station."station_id" = $status."station_id" 
    and $status."num_bikes_available" > $status."num_docks_available" 
    and $station."region_id" != "region_80" 
    return {"station_id": $station."station_id", "region_id": $station."region_id"} 
return [$result]

Took: 0.08336520195007324 ms
[]


- Query 6 without optimization

In [138]:
f = open('Data/station-information.json', )
data = json.load(f).get('data').get('stations')
station = json_datamodel(data, "station")
f.close()
f = open('Data/station-status.json', )
data = json.load(f).get('data').get('stations')
status = json_datamodel(data, "status")
f.close()

query_6 = "let $result := for $station in $station[], $status in $status[] where $station.'station_id' = $status.'station_id' and $status.'num_bikes_available' > $status.'num_docks_available' and $station.'region_id' != 'region_80' return {'station_id': $station.'station_id', 'region_id': $station.'region_id'} return [$result]"
result_6 = execution_without_optimization(query_6)
print("OUTPUT QUERY 6 WITHOUT OPTIMIZATION: ", result_6)
print("LENGTH OUTPUT QUERY 6 WITHOUT OPTIMIZATION: ", len(result_6))

Took: 2.6799161434173584 ms
OUTPUT QUERY 6 WITHOUT OPTIMIZATION:  [{'station_id': 'hub_2456', 'region_id': 'region_81'}, {'station_id': 'hub_2457', 'region_id': 'region_81'}, {'station_id': 'hub_2459', 'region_id': 'region_81'}, {'station_id': 'hub_2460', 'region_id': 'region_81'}, {'station_id': 'hub_2528', 'region_id': 'region_81'}, {'station_id': 'hub_2529', 'region_id': 'region_81'}, {'station_id': 'hub_2555', 'region_id': 'region_81'}, {'station_id': 'hub_2606', 'region_id': 'region_81'}, {'station_id': 'hub_2609', 'region_id': 'region_81'}, {'station_id': 'hub_2610', 'region_id': 'region_81'}, {'station_id': 'hub_3583', 'region_id': 'region_81'}, {'station_id': 'hub_3779', 'region_id': 'region_81'}]
LENGTH OUTPUT QUERY 6 WITHOUT OPTIMIZATION:  12


- Query 6 with optimization

In [139]:
f = open('Data/station-information.json', )
data = json.load(f).get('data').get('stations')
station = json_datamodel(data, "station")
f.close()
f = open('Data/station-status.json', )
data = json.load(f).get('data').get('stations')
status = json_datamodel(data, "status")
f.close()

query_6 = "let $result := for $station in $station[], $status in $status[] where $station.'station_id' = $status.'station_id' and $status.'num_bikes_available' > $status.'num_docks_available' and $station.'region_id' != 'region_80' return {'station_id': $station.'station_id', 'region_id': $station.'region_id'} return [$result]"
result_6 = execution(query_6, True)
print("OUTPUT QUERY 6 WITH OPTIMIZATION: ", result_6)
print("LENGTH OUTPUT QUERY 6 WITH OPTIMIZATION: ", len(result_6))

Took: 0.4665720462799072 ms
OUTPUT QUERY 6 WITH OPTIMIZATION:  [{'station_id': 'hub_2456', 'region_id': 'region_81'}, {'station_id': 'hub_2457', 'region_id': 'region_81'}, {'station_id': 'hub_2459', 'region_id': 'region_81'}, {'station_id': 'hub_2460', 'region_id': 'region_81'}, {'station_id': 'hub_2528', 'region_id': 'region_81'}, {'station_id': 'hub_2529', 'region_id': 'region_81'}, {'station_id': 'hub_2555', 'region_id': 'region_81'}, {'station_id': 'hub_2606', 'region_id': 'region_81'}, {'station_id': 'hub_2609', 'region_id': 'region_81'}, {'station_id': 'hub_2610', 'region_id': 'region_81'}, {'station_id': 'hub_3583', 'region_id': 'region_81'}, {'station_id': 'hub_3779', 'region_id': 'region_81'}]
LENGTH OUTPUT QUERY 6 WITH OPTIMIZATION:  12


## 4. Rick and Morty 

**Use Case 7**: *Find all episode names where at least one of the characters is neither female nor male* 

- Parse query in JSONiq

In [32]:
%%rumble 
let $character := []
let $episode := []
let $result := for $episode in $episode[], $character in $character[] where $episode.characters."url" = $character."url" where $character."gender" != "Male" and not $character."gender" =  "Female" return {"name": $episode."episode"} return [$result]

Took: 0.09275698661804199 ms
[]


- Query 7 without optimization

In [143]:
f = open('Data/characters.json', )
data = json.load(f)
character = json_datamodel(data, "character")
f.close()
f = open('Data/episodes.json', )
data = json.load(f)
episode = json_datamodel(data, "episode")
f.close()
f = open('Data/locations.json', )
data = json.load(f)
location = json_datamodel(data, "location")
f.close()

query_7 = "let $result := for $episode in $episode[] for $character in $character[] where $episode.'characters' = $character.'url' where $character.'gender' != 'Male' and not $character.'gender' = 'Female' return {'name': $episode.'episode'} return [$result]"
result_7 = execution_without_optimization(query_7)
print("OUTPUT QUERY 7: ")
print(result_7)
print("LENGTH OUTPUT QUERY 7: ", len(result_7))  

Took: 9.406656980514526 ms
OUTPUT QUERY 7: 
[{'episode': 'S01E01'}, {'episode': 'S01E01'}, {'episode': 'S01E01'}, {'episode': 'S01E01'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S01E05'}, {'episode': 'S01E08'}, {'episode': 'S01E08'}, {'episode': 'S01E08'}, {'episode': 'S01E08'}, {'episode': 'S01E08'}, {'episode': 'S01E08'}, {'episode': 'S01E10'}, {'episode': 'S01E10'}, {'episode': 'S01E11'}, {'episode': 'S01E11'}, {'episode': 'S01E11'}, {'episode': 'S01E11'}, {'episode': 'S01E11'}, {'episode': 'S01E11'}, {'episode': 'S01E11'}, {'episode': 'S02E01'}, {'episode': 'S02E02'}, {'episode': 'S02E02'}, {'episode': 'S02E03'}, {'episode': 'S02E03'}, {'episode': 'S02E04'}, {'episode': 'S02E04'}, {'episode': 'S02E05'}, {'episode': 'S02E05'}, {'episode': 'S02E05'}, {'episode': 'S02E05'}, {'episode': 'S02E06'}, {'episode': 'S02E06'}, {'episode': 'S02E07'}, {'episode': 'S02E08'}, {'episode': 

- Query 7 with optimization

In [144]:
f = open('Data/characters.json', )
data = json.load(f)
character = json_datamodel(data, "character")
f.close()
f = open('Data/episodes.json', )
data = json.load(f)
episode = json_datamodel(data, "episode")
f.close()
f = open('Data/locations.json', )
data = json.load(f)
location = json_datamodel(data, "location")
f.close()

query_7 = "let $result := for $episode in $episode[] for $character in $character[] where $episode.'characters' = $character.'url' where $character.'gender' != 'Male' and not $character.'gender' = 'Female' return {'name': $episode.'episode'} return [$result]"
result_7 = execution(query_7, True)
print("OUTPUT QUERY 7: ")
print(result_7)
print("LENGTH OUTPUT QUERY 7: ", len(result_7))  

Took: 0.7864601612091064 ms
OUTPUT QUERY 7: 
[{'episode': 'S03E10'}, {'episode': 'S01E01'}, {'episode': 'S01E11'}, {'episode': 'S02E08'}, {'episode': 'S03E04'}, {'episode': 'S02E03'}, {'episode': 'S02E08'}, {'episode': 'S02E07'}, {'episode': 'S02E10'}, {'episode': 'S02E01'}, {'episode': 'S03E01'}, {'episode': 'S02E10'}, {'episode': 'S01E10'}, {'episode': 'S03E01'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S01E03'}, {'episode': 'S03E04'}, {'episode': 'S03E04'}, {'episode': 'S02E08'}, {'episode': 'S02E04'}, {'episode': 'S03E05'}, {'episode': 'S01E08'}, {'episode': 'S02E08'}, {'episode': 'S01E08'}, {'episode': 'S01E01'}, {'episode': 'S03E03'}, {'episode': 'S02E10'}, {'episode': 'S02E05'}, {'episode': 'S02E08'}, {'episode': 'S02E10'}, {'episode': 'S02E05'}, {'episode': 'S02E04'}, {'episode': 'S01E08'}, {'episode': 'S02E06'}, {'episode': 'S03E03'}, {'episode': 'S02E08'}, {'episode': 'S01E05'}, {'episode':

**Use Case 8**: *Find all character names, episode names and location types of charachters that die and are of type human in a specific episode* 

- Parse query in JSONiq

In [145]:
%%rumble 
let $character := []
let $episode := []
let $location := []
let $result := for $episode in $episode[], $character in $character[] 
        where $episode."characters" = $character."url" 
        for $location in $location[] where $character.location."name"=$location."name" 
        where $character."species" = "Human" and $character."status"="Dead" 
        return {"name": $character."name", "episode":$episode."episode", "location":$location."name"} 
return [$result]

Took: 0.07622289657592773 ms
[]


- Query 8 without optimization 

In [45]:
f = open('Data/characters.json', )
data = json.load(f)
character = json_datamodel(data, "character")
f.close()
f = open('Data/episodes.json', )
data = json.load(f)
episode = json_datamodel(data, "episode")
f.close()
f = open('Data/locations.json', )
data = json.load(f)
location = json_datamodel(data, "location")
f.close()


query_8 = "let $result := for $episode in $episode[], $character in $character[] where $episode.'characters' = $character.'url' for $location in $location[] where $character.location.'name'=$location.'name' where $character.'species' = 'Human' and $character.'status'='Dead' return {'name': $character.'name', 'episode':'$episode.'episode', 'location':$location.'name'} return [$result]"
result_8 = execution_without_optimization(query_8)
print("OUTPUT QUERY 8: ")
print(result_8)
print("LENGTH OUTPUT QUERY 8: ", len(result_8))  

Took: 3516.147355079651 ms
OUTPUT QUERY 8: 
[{'episode': 'S01E01', 'name': 'Earth (C-137)'}, {'episode': 'S01E01', 'name': 'Earth (C-137)'}, {'episode': 'S01E03', 'name': 'Anatomy Park'}, {'episode': 'S01E03', 'name': 'Anatomy Park'}, {'episode': 'S01E03', 'name': 'Anatomy Park'}, {'episode': 'S01E03', 'name': 'Earth (C-137)'}, {'episode': 'S01E05', 'name': 'Earth (C-137)'}, {'episode': 'S01E05', 'name': 'Earth (C-137)'}, {'episode': 'S01E05', 'name': 'Earth (C-137)'}, {'episode': 'S01E06', 'name': 'Earth (C-137)'}, {'episode': 'S01E06', 'name': 'Earth (Replacement Dimension)'}, {'episode': 'S01E06', 'name': 'Earth (Replacement Dimension)'}, {'episode': 'S01E07', 'name': 'Earth (Replacement Dimension)'}, {'episode': 'S01E08', 'name': 'Interdimensional Cable'}, {'episode': 'S01E10', 'name': 'Citadel of Ricks'}, {'episode': 'S01E10', 'name': 'Citadel of Ricks'}, {'episode': 'S01E10', 'name': "Earth (Evil Rick's Target Dimension)"}, {'episode': 'S01E10', 'name': 'Citadel of Ricks'}, {'epi

- Query 8 with optimization

In [33]:
f = open('Data/characters.json', )
data = json.load(f)
character = json_datamodel(data, "character")
f.close()
f = open('Data/episodes.json', )
data = json.load(f)
episode = json_datamodel(data, "episode")
f.close()
f = open('Data/locations.json', )
data = json.load(f)
location = json_datamodel(data, "location")
f.close()

query_8 = "let $result := for $episode in $episode[], $character in $character[] where $episode.'characters' = $character.'url' for $location in $location[] where $character.location.'name'=$location.'name' where $character.'species' = 'Human' and $character.'status'='Dead' return {'name': $character.'name', 'episode':'$episode.'episode', 'location':$location.'name'} return [$result]"
result_8 = execution(query_8, True)
print("OUTPUT QUERY 8: ")
print(result_8)
print("LENGTH OUTPUT QUERY 8: ", len(result_8)) 

Took: 2.5363292694091797 ms
OUTPUT QUERY 8: 
[{'episode': 'S01E01', 'name': 'Earth (C-137)'}, {'episode': 'S01E01', 'name': 'Earth (C-137)'}, {'episode': 'S01E03', 'name': 'Anatomy Park'}, {'episode': 'S01E03', 'name': 'Anatomy Park'}, {'episode': 'S01E03', 'name': 'Anatomy Park'}, {'episode': 'S01E03', 'name': 'Earth (C-137)'}, {'episode': 'S01E05', 'name': 'Earth (C-137)'}, {'episode': 'S01E05', 'name': 'Earth (C-137)'}, {'episode': 'S01E05', 'name': 'Earth (C-137)'}, {'episode': 'S01E06', 'name': 'Earth (C-137)'}, {'episode': 'S01E06', 'name': 'Earth (Replacement Dimension)'}, {'episode': 'S01E06', 'name': 'Earth (Replacement Dimension)'}, {'episode': 'S01E07', 'name': 'Earth (Replacement Dimension)'}, {'episode': 'S01E08', 'name': 'Interdimensional Cable'}, {'episode': 'S01E10', 'name': 'Citadel of Ricks'}, {'episode': 'S01E10', 'name': 'Citadel of Ricks'}, {'episode': 'S01E10', 'name': "Earth (Evil Rick's Target Dimension)"}, {'episode': 'S01E10', 'name': 'Citadel of Ricks'}, {'ep

## 5. Nobel Prize Winners 

**Use Case 9**: *The retrieval of the year the Nobel Prize was received, and the first name and surname of all Dutch, German, and Belgium Nobel Prize Laureates that won a Nobel Prize in the category economics, while sorting the result* 

- Parse query in JSONiq

In [168]:
%%rumble 
let $prize := []
let $laureate := []
let $country := []
let $result := for $prize in $prize[] 
    for $laureate in $laureate[] 
    where $prize.laureates."id"=$laureate."id" and $prize."category" = "economics" 
    where $laureate."bornCountryCode"="NL" or $laureate."bornCountryCode" = "DE" or $laureate."bornCountryCode"="BE" 
    order by $prize."year" descending, $laureate."surname" ascending  
    return {"year":$prize."year", "firstname": $laureate."firstname", "surname": $laureate."surname"} 
return [$result]

Took: 0.0864722728729248 ms
[]


- Query 9 without optimization

In [44]:
f = open('Data/Prizes.json', )
data = json.load(f).get("prizes")
prize = json_datamodel(data, "prize")
f.close()
f = open('Data/Laureates.json', )
data = json.load(f).get("laureates")
laureate = json_datamodel(data, "laureate")
f.close()

query_9 = "let $result := for $prize in $prize[] for $laureate in $laureate[] where $prize.laureates.'id'=$laureate.'id' and $prize.'category' = 'economics' where $laureate.'bornCountryCode'='NL' or $laureate.'bornCountryCode' = 'DE' or $laureate.'bornCountryCode'='BE' order by $prize.'year' descending, $laureate.'surname' ascending  return {'year':$prize.'year', 'firstname': $laureate.'firstname', 'surname': $laureate.'surname'} return [$result]"
result_9 = execution_without_optimization(query_9)
print("OUTPUT QUERY 9: ")
print(result_9)
print("LENGTH OUTPUT QUERY 9: ", len(result_9))

Took: 522.3089599609375 ms
OUTPUT QUERY 9: 
[{'year': '2005', 'firstname': 'Robert J.', 'surname': 'Aumann'}, {'year': '1975', 'firstname': 'Tjalling C.', 'surname': 'Koopmans'}, {'year': '1969', 'firstname': 'Jan', 'surname': 'Tinbergen'}]
LENGTH OUTPUT QUERY 9:  3


- Query 9 with optimization

In [43]:
f = open('Data/Prizes.json', )
data = json.load(f).get("prizes")
prize = json_datamodel(data, "prize")
f.close()
f = open('Data/Laureates.json', )
data = json.load(f).get("laureates")
laureate = json_datamodel(data, "laureate")
f.close()

query_9 = "let $result := for $prize in $prize[] for $laureate in $laureate[] where $prize.laureates.'id'=$laureate.'id' and $prize.'category' = 'economics' where $laureate.'bornCountryCode'='NL' or $laureate.'bornCountryCode' = 'DE' or $laureate.'bornCountryCode'='BE' order by $prize.'year' descending, $laureate.'surname' ascending  return {'year':$prize.'year', 'firstname': $laureate.'firstname', 'surname': $laureate.'surname'} return [$result]"
result_9 = execution(query_9, True)
print("OUTPUT QUERY 9: ")
print(result_9)
print("LENGTH OUTPUT QUERY 9: ", len(result_9))

Took: 2.0986149311065674 ms
OUTPUT QUERY 9: 
[{'firstname': 'Robert J.', 'surname': 'Aumann', 'year': '2005'}, {'firstname': 'Tjalling C.', 'surname': 'Koopmans', 'year': '1975'}, {'firstname': 'Jan', 'surname': 'Tinbergen', 'year': '1969'}]
LENGTH OUTPUT QUERY 9:  3


**Use Case 10**: *Retrieve the names of all Dutch Nobel Prize winners* 

- Parse query in JSONiq

In [42]:
%%rumble 
let $prize := []
let $laureate:= []
let $country := []
let $result := for $prize in $prize[] 
        for $laureate in $laureate[]  
        where $prize."laureates"."id" = $laureate."id"  
        for $country in $country[] 
        where $laureate."bornCountryCode"= $country."code" and $country."name"="the Netherlands" 
        return {"firstname": $laureate."firstname", "surname": $laureate."surname"} 
    return [$result]

Took: 0.07467889785766602 ms
[]


- Query 10 without optimization

In [35]:
f = open('Data/Prizes.json', )
data = json.load(f).get("prizes")
prize = json_datamodel(data, "prize")
f.close()
f = open('Data/Laureates.json', )
data = json.load(f).get("laureates")
laureate = json_datamodel(data, "laureate")
f.close()
f = open('Data/Countries.json', )
data = json.load(f).get("countries")
country = json_datamodel(data, "country")
f.close()

# DO NOT RUN THIS QUERY AS IT WILL TAKE VERY AND COULD CRASH THIS NOTEBOOK
query_10 = "let $result := for $prize in $prize[] for $laureate in $laureate[]  where $prize.laureates.'id'=$laureate.'id' for $country in $country[] where $laureate.'bornCountryCode'= $country.'code' and $country.'name'='the Netherlands' return {'firstname': $laureate.'firstname', 'surname': $laureate.'surname'} return [$result]"
#result_10 = execution(query_10, False)
result_10 = ""
print("OUTPUT QUERY 10: ")
print(result_10)
print("LENGTH OUTPUT QUERY 10: ", len(result_10))               

OUTPUT QUERY 10: 

LENGTH OUTPUT QUERY 10:  0


- Query 10 with optimization

In [38]:
f = open('Data/Prizes.json', )
data = json.load(f).get("prizes")
prize = json_datamodel(data, "prize")
f.close()
f = open('Data/Laureates.json', )
data = json.load(f).get("laureates")
laureate = json_datamodel(data, "laureate")
f.close()
f = open('Data/Countries.json', )
data = json.load(f).get("countries")
country = json_datamodel(data, "country")
f.close()

query_10 = "let $result := for $prize in $prize[] for $laureate in $laureate[]  where $prize.laureates.'id'=$laureate.'id' for $country in $country[] where $laureate.'bornCountryCode'= $country.'code' and $country.'name'='the Netherlands' return {'firstname': $laureate.'firstname', 'surname': $laureate.'surname'} return [$result]"
result_10 = execution(query_10, True)
print("OUTPUT QUERY 10: ")
print(result_10)
print("LENGTH OUTPUT QUERY 10: ", len(result_10))  

Took: 2.4567532539367676 ms
OUTPUT QUERY 10: 
[{'firstname': 'Hendrik A.', 'surname': 'Lorentz'}, {'firstname': 'Pieter', 'surname': 'Zeeman'}, {'firstname': 'Johannes Diderik', 'surname': 'van der Waals'}, {'firstname': 'Heike', 'surname': 'Kamerlingh Onnes'}, {'firstname': 'Frits', 'surname': 'Zernike'}, {'firstname': 'Nicolaas', 'surname': 'Bloembergen'}, {'firstname': 'Simon', 'surname': 'van der Meer'}, {'firstname': 'Gerardus', 'surname': "'t Hooft"}, {'firstname': 'Martinus J.G.', 'surname': 'Veltman'}, {'firstname': 'Jacobus H.', 'surname': "van 't Hoff"}, {'firstname': 'Peter', 'surname': 'Debye'}, {'firstname': 'Paul J.', 'surname': 'Crutzen'}, {'firstname': 'Christiaan', 'surname': 'Eijkman'}, {'firstname': 'Nikolaas', 'surname': 'Tinbergen'}, {'firstname': 'Tobias', 'surname': 'Asser'}, {'firstname': 'Jan', 'surname': 'Tinbergen'}, {'firstname': 'Tjalling C.', 'surname': 'Koopmans'}, {'firstname': 'Bernard L.', 'surname': 'Feringa'}]
LENGTH OUTPUT QUERY 10:  18


END NOTEBOOK 